In [3]:
import os
import random
import numpy as np
import hdbscan
import torch
import clip
from PIL import Image

In [4]:
# `umap-learn` 패키지가 제대로 로드되지 않는 경우를 대비한 경로 추가
import sys
sys.path.append(r"C:\Users\user\anaconda3\envs\my_new_env\lib\site-packages")

In [6]:
folder_path =r"E:\AI_HUB\data\Training\raw_data\TS_photo"
save_path = r"E:\AI_HUB\data\Training\raw_data\TS_photo\features"

os.makedirs(save_path, exist_ok=True)

In [9]:
# 파일 경로 생성
features_path = os.path.join(save_path, "clip_features.npy")
image_names_path = os.path.join(save_path, "clip_image_names.npy")

In [27]:
# ✅ 이미지 로드 함수
def load_images(folder_path, limit=3000):
    image_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_files.append(os.path.join(root, file))
    return random.sample(image_files, limit) if len(image_files) > limit else image_files

In [28]:
# ✅ CLIP 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [29]:
# ✅ 이미지 특징 벡터 추출 함수
def extract_features_clip(image_path):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model.encode_image(image)
    return features.cpu().numpy().flatten()

In [30]:
# # ✅ 3000장 랜덤 이미지 선택 및 특징 추출
# image_files = load_images(folder_path, limit=3000)
# features = np.array([extract_features_clip(img) for img in image_files])
# image_names = [os.path.basename(img) for img in image_files]

KeyboardInterrupt: 

In [16]:
# # ✅ 특징 벡터 저장
# np.save(os.path.join(save_path, "clip_features.npy"), features)
# np.save(os.path.join(save_path, "clip_image_names.npy"), np.array(image_names))

In [14]:
# 저장된 NumPy 배열 불러오기
features = np.load(features_path)
image_names = np.load(image_names_path)

In [5]:
import umap
print(umap.__file__)

C:\Users\user\anaconda3\envs\my_new_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


C:\Users\user\anaconda3\envs\my_new_env\lib\site-packages\umap\__init__.py


In [48]:
import importlib
umap = importlib.import_module("umap")
print(umap)

<module 'umap' (namespace) from ['C:\\Users\\user\\anaconda3\\Lib\\site-packages\\umap']>


In [23]:
from umap import UMAP  # 올바른 임포트

# UMAP 차원 축소 (2D) - n_neighbors 줄이기
umap_2d = UMAP(
    n_components=2,
    n_neighbors=20,
    min_dist=0.2,
    random_state=42
)

features_2d = umap_2d.fit_transform(features)

C:\Users\user\anaconda3\envs\my_new_env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\user\anaconda3\envs\my_new_env\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [24]:
# HDBSCAN 클러스터링 적용 - min_cluster_size 조정
clusterer = hdbscan.HDBSCAN(min_cluster_size=10, 
                            min_samples=3, 
                            metric="manhattan")
labels = clusterer.fit_predict(features_2d)

C:\Users\user\anaconda3\envs\my_new_env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\user\anaconda3\envs\my_new_env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [26]:
# 클러스터링 결과 저장
np.save(os.path.join(save_path, "cluster_labels.npy"), labels)

# 클러스터 개수 출력
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"📌 생성된 클러스터 개수: {n_clusters}")

📌 생성된 클러스터 개수: 2
